In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llama-3.2/transformers/1b/1/config.json
/kaggle/input/llama-3.2/transformers/1b/1/README.md
/kaggle/input/llama-3.2/transformers/1b/1/USE_POLICY.md
/kaggle/input/llama-3.2/transformers/1b/1/tokenizer.json
/kaggle/input/llama-3.2/transformers/1b/1/tokenizer_config.json
/kaggle/input/llama-3.2/transformers/1b/1/LICENSE.txt
/kaggle/input/llama-3.2/transformers/1b/1/model.safetensors
/kaggle/input/llama-3.2/transformers/1b/1/special_tokens_map.json
/kaggle/input/llama-3.2/transformers/1b/1/.gitattributes
/kaggle/input/llama-3.2/transformers/1b/1/generation_config.json


In [2]:
%%capture

# Install and update all the necessary Python packages
%pip install -U transformers 
%pip install -U datasets 
%pip install -U accelerate 
%pip install -U peft 
%pip install -U trl 
%pip install -U bitsandbytes 
%pip install -U wandb

In [3]:
# Load the Python packages and functions for fine-tuning and evaluation
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch, wandb
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

In [4]:
wb_token = user_secrets.get_secret("wandb")
wandb.login(key=wb_token)

run = wandb.init(project='try-connect-db', 
    job_type="training", 
    anonymous="allow",
    settings=wandb.Settings(start_method="fork"))

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: overtheskyy (overtheskyy-workspaces). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.5
wandb: Run data is saved locally in /kaggle/working/wandb/run-20241030_200648-vkg94knw
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run unraveling-riddle-7
wandb: ⭐️ View project at https://wandb.ai/overtheskyy-workspaces/try-connect-db
wandb: 🚀 View run at https://wandb.ai/overtheskyy-workspaces/try-connect-db/runs/vkg94knw
